In [1]:
import os
import random
from PIL import Image
from keras import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.models import model_from_json

In [ ]:
# !python -m pip install --upgrade pip

In [2]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
mode= "categorical"
batch_size = 32
img_size = 100
# random_seed = 20

train_generator = train_datagen.flow_from_directory('dataset/train', class_mode=mode, batch_size=batch_size, target_size=(img_size,img_size))
val_generator = val_datagen.flow_from_directory('dataset/val', class_mode=mode, batch_size=batch_size, target_size=(img_size,img_size))


Found 3591 images belonging to 2 classes.
Found 1197 images belonging to 2 classes.


In [3]:
def create_model_net():
    kernel_size = (3,3)
    pool_size = (2,2)
    first_filters = 32
    second_filters = 64
    third_filters = 128
    
    dropout_conv = 0.3
    dropout_dense = 0.3
    
    model = Sequential()
    
    model.add(Conv2D(first_filters,kernel_size,activation='relu',padding='same',input_shape=(img_size,img_size,3)))
    model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(dropout_conv))    

    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(dropout_conv))

    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(dropout_conv))

    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(dropout_conv))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(dropout_conv))
    
    model.add(Flatten())
    model.add(Dense(units=256,activation="relu"))
    model.add(Dropout(dropout_dense))
    model.add(Dense(2, activation="softmax"))
       
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    
    return model

In [4]:
model = create_model_net()

In [5]:
model.fit_generator(generator = train_generator,steps_per_epoch=256, validation_data= val_generator, validation_steps=50, epochs=5)


c:\users\94766\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
256/256 [==============================] - 486s 2s/step - loss: 0.7600 - accuracy: 0.6185 - val_loss: 0.1355 - val_accuracy: 0.9405
Epoch 2/5
256/256 [==============================] - 427s 2s/step - loss: 0.1427 - accuracy: 0.9470 - val_loss: 0.0836 - val_accuracy: 0.9823
Epoch 3/5
256/256 [==============================] - 491s 2s/step - loss: 0.0841 - accuracy: 0.9761 - val_loss: 0.0762 - val_accuracy: 0.9827
Epoch 4/5
256/256 [==============================] - 427s 2s/step - loss: 0.0677 - accuracy: 0.9775 - val_loss: 0.0858 - val_accuracy: 0.9722
Epoch 5/5
256/256 [==============================] - 424s 2s/step - loss: 0.0597 - accuracy: 0.9851 - val_loss: 0.1139 - val_accuracy: 0.9652


In [6]:
if not os.path.exists('models'):
    os.mkdir('models')
# model.save('models/model.h5')

# serialize model to JSON
model_json = model.to_json()
with open("model5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model5.h5")
print("Saved model to disk")

Saved model to disk


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 64)        0

In [11]:
for layer in model.layers:
    weights = layer.get_weights()
    print(weights)

[array([[[[ 2.23101140e-03,  2.15957128e-02, -1.21261299e-01,
           7.95175582e-02,  1.30664855e-02,  8.84589851e-02,
           1.75731584e-01, -1.05336830e-01, -7.14869872e-02,
           6.60181120e-02,  7.85353333e-02, -2.23071985e-02,
           1.23378383e-02, -1.36901587e-01, -9.40300301e-02,
          -3.34463492e-02,  1.42581984e-01,  9.90430191e-02,
          -1.01497628e-01, -1.04943535e-03,  2.96223331e-02,
           3.94319221e-02,  7.82983452e-02, -1.04624867e-01,
          -4.36259508e-02, -1.63273513e-02,  9.38522369e-02,
          -5.13637345e-03, -7.77853876e-02,  8.81011188e-02,
           6.77361339e-02,  3.71429026e-02],
         [ 4.11823392e-03, -6.63513243e-02,  2.32140124e-02,
          -9.12526436e-03, -1.76995583e-02,  4.41935472e-02,
          -1.11778483e-01,  5.55067509e-02,  4.82899770e-02,
           8.17311835e-03, -1.37948215e-01, -1.07426293e-01,
           4.47247140e-02, -1.40069649e-01, -1.37201354e-01,
          -1.07168756e-01, -8.59720781e

In [12]:
print(weights)

[array([[-3.03801838e-02, -4.71427850e-02],
       [-8.51917341e-02, -4.19938788e-02],
       [ 7.52686933e-02,  9.14563090e-02],
       [-1.35451183e-01, -7.73380473e-02],
       [ 8.41277242e-02,  4.93255407e-02],
       [ 3.30722332e-02,  5.70487492e-02],
       [-1.08401202e-01, -7.87484720e-02],
       [-4.28246297e-02, -1.28394410e-01],
       [-1.66523457e-02,  2.30514370e-02],
       [-4.68351953e-02, -2.81859115e-02],
       [-6.39528558e-02, -5.65091334e-03],
       [-3.49731520e-02,  2.88631655e-02],
       [ 1.31820276e-01,  7.63040259e-02],
       [ 3.80856395e-02,  6.28282130e-02],
       [-1.19750304e-02, -1.40939415e-01],
       [-3.31728533e-03, -3.20753269e-02],
       [-1.09198421e-01, -2.81954594e-02],
       [ 8.59254040e-03, -2.32950673e-02],
       [ 1.06991954e-01, -1.14015393e-01],
       [ 2.05445606e-02,  3.57389376e-02],
       [ 3.93375084e-02,  9.75245684e-02],
       [-4.61441539e-02,  1.79314464e-02],
       [ 9.41127166e-02, -1.91032350e-01],
       [-3

In [15]:
!pip3 install pydot

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [20]:
from keras.utils.vis_utils import plot_model
#from keras.utils import plot_model

In [18]:
from keras.utils import plot_model
plot_model(model, to_file='/ramz/model.png', show_shapes=True,)

ImportError: cannot import name 'plot_model' from 'keras.utils' (c:\users\94766\appdata\local\programs\python\python37\lib\site-packages\keras\utils\__init__.py)